In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU


In [ ]:
file_path = '/content/Drying3.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

X = data[['Drying Time (min )', 'Velocity m/s', 'Temperatures C', 'Thickness mm', 'Sample mass in (g)']]
y = data[['DR(g/min)']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model_8_layers = Sequential([
    Dense(64, input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(y_train.shape[1], activation='linear')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
reduce_lr_8 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-6)
early_stop_8 = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_8_layers.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])


In [ ]:

history = model_8_layers.fit(X_train_scaled, y_train,
                    epochs=1000,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1,
                   )

y_pred = model_8_layers.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)

model_8_layers.save('final_dnn_model.keras')



Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - loss: 675.0029 - mae: 19.9455 - val_loss: 477.7534 - val_mae: 16.0053
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 681.0563 - mae: 20.2885 - val_loss: 468.4587 - val_mae: 15.9167
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 557.5815 - mae: 18.5989 - val_loss: 458.2936 - val_mae: 15.8183
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 561.5216 - mae: 18.7917 - val_loss: 446.3714 - val_mae: 15.6879
Epoch 5/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 544.7729 - mae: 18.1692 - val_loss: 434.2109 - val_mae: 15.5515
Epoch 6/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 518.4783 - mae: 18.2589 - val_loss: 421.3170 - val_mae: 15.4013
Epoch 7/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 534.1987 - mae: 19.0333 - val_loss: 406.4744 - val_mae: 15.2140
Epoch 8/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 491.3784 - mae: 18.1540 - val_loss: 390.6193 - val_mae: 15.0094
Epoch 9/1000
7/7 ━━━━

In [6]:
import numpy as np

def predict_new(data):
    data_scaled = scaler.transform([data])
    prediction = model_8_layers.predict(data_scaled)
    return [f"{val:.4f}" for val in prediction[0]]  # تحويل القيم إلى نص وتنسيقها لأربعة أرقام عشرية

# بيانات جديدة للتنبؤ
data_example = [210, 2, 80, 7, 488.16]
prediction = predict_new(data_example)

# عرض النتائج
print("Predictions for new data:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predictions for new data: ['20.8237']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
